In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Human_Data.zip

# Preprocess


## Data getting

Get the data from drive, which was downloaded from OpenImages

In [ ]:
import subprocess
import os

os.makedirs("/content/Data")
def unrar_files():
    destination_folder = "/content/Data"
    for file_name in ["train.rar", 'test.rar', 'val.rar']:
      unrar_command = f"unrar x {'/content/drive/MyDrive/CV_Dataset/'+file_name} {destination_folder}"
      subprocess.run(unrar_command, shell=True)

unrar_files()

In [ ]:
def list_files(folder_path):
    file_list = []
    for file_name in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file_name)):
            file_list.append(file_name[:-4])
    return file_list

Take only 1000 images from each test and validation dataset. Take 8000 images from train dataset.

In [ ]:
for file_type in ["test", "val"]:
    files = list_files("/content/Data/"+file_type)
    for file_name in files[1000:]:
        os.remove("/content/Data/" + file_type + '/' + file_name + '.jpg')
files = list_files("/content/Data/train")
for file_name in files[8000:]:
    os.remove("/content/Data/" + 'train' + '/' + file_name + '.jpg')

Remove unmatch data like image without annotations and otherwise

In [ ]:
import pandas as pd


def data_unite():
    for file_type in ["test", "train", "val"]:
        files = list_files("/content/Data/"+file_type)
        bb_anno =  pd.read_csv('/content/drive/MyDrive/CV_Dataset/Bounding Boxes/'+file_type+'.csv')
        image_list = bb_anno["ImageID"].to_list()
        for file_name in files:
          if file_name not in image_list:
            os.remove("/content/Data/" + file_type + '/' + file_name + '.jpg')
        tmp = []
        files = list_files("/content/Data/"+file_type)
        for image in image_list:
          if image in files:
            tmp.append(image)
        df = bb_anno[bb_anno['ImageID'].isin(tmp)]
        df.to_csv("/content/Data/"+file_type+".csv")


data_unite()



## Format changing

We change the format of the annotation data from coco format to Yolo format

In [ ]:
import os
import pandas as pd

In [ ]:
for file_type in ['train', 'test', 'validation']:
    dataframe = pd.read_csv("/content/"+file_type+'.csv')
    images = dataframe["ImageID"].unique()

    for id in images:
    # Filter the dataframe based on the ID
      filtered_df = dataframe[dataframe['ImageID'] == id]

      # Open a text file for writing
      filename = "/content/labels/" + file_type + f'/{id}.txt'
      with open(filename, 'w') as file:
          # Iterate over the filtered dataframe rows
          for _, row in filtered_df.iterrows():
              # Calculate the center, width, and height
              x_center = (row['XMin'] + row['XMax']) / 2
              y_center = (row['YMin'] + row['YMax']) / 2
              width = row['XMax'] - row['XMin']
              height = row['YMax'] - row['YMin']

              # Write the object information to the text file
              file.write(f"0 {x_center} {y_center} {width} {height}\n")

## Image resizing and normalizing

The image size of image data is vary in many size, so we resize the image to 1024x1024, also we normalized the image.

In [ ]:
import cv2

files = list_files('/content/Data/test')
for file in files:
  image = cv2.imread('/content/Data/test/'+file+'.jpg')
  print(image.shape)

In [ ]:
import cv2

def resize_and_normalize_image(image_path, output_path, target_size):
    # Read the image
    image = cv2.imread(image_path)

    # Resize the image to the target size
    resized_image = cv2.resize(image, target_size)

    # Normalize the image (optional)
    normalized_image = cv2.normalize(resized_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Remove the old image
    os.remove(image_path)

    # Store the new image
    cv2.imwrite(output_path, normalized_image)

for file_type in ["test", "train", "val"]:
    files = list_files('/content/Data/'+file_type)
    target_size = (1024,1024)
    for file in files:
        resize_and_normalize_image('/content/Data/'+file_type+'/'+file+'.jpg', '/content/Data/'+file_type+'/'+file+'.jpg', target_size)

